# About This Notebook

This notebook shows how to implement **Low-Rank Tensor Completion with Truncated Nuclear Norm minimization (LRTC-TNN)** on some real-world data sets. For an in-depth discussion of LRTC-TNN, please see our article [1].

<div class="alert alert-block alert-info">
<font color="black">
<b>[1]</b> Xinyu Chen, Jinming Yang, Lijun Sun (2020). <b>A Nonconvex Low-Rank Tensor Completion Model for Spatiotemporal Traffic Data Imputation</b>. arXiv.2003.10271. <a href="https://arxiv.org/abs/2003.10271" title="PDF"><b>[PDF]</b></a> 
</font>
</div>


## Quick Run

This notebook is publicly available at [https://github.com/xinychen/tensor-learning](https://github.com/xinychen/tensor-learning).


## Low-Rank Tensor Completion

We start by importing the necessary dependencies.

In [1]:
import numpy as np
from numpy.linalg import inv as inv

### Tensor Unfolding (`ten2mat`) and Matrix Folding (`mat2ten`)

Using numpy reshape to perform 3rd rank tensor unfold operation. [[**link**](https://stackoverflow.com/questions/49970141/using-numpy-reshape-to-perform-3rd-rank-tensor-unfold-operation)]

In [2]:
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, tensor_size, mode):
    index = list()
    index.append(mode)
    for i in range(tensor_size.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(tensor_size[index]), order = 'F'), 0, mode)

### Singular Value Thresholding (SVT) for TNN

In [3]:
def svt_tnn(mat, alpha, rho, theta):
    """This is a Numpy dependent singular value thresholding (SVT) process."""
    u, s, v = np.linalg.svd(mat, full_matrices = False)
    vec = s.copy()
    vec[theta :] = s[theta :] - alpha / rho
    vec[vec < 0] = 0
    return np.matmul(np.matmul(u, np.diag(vec)), v)

**Potential alternative for this**:

This is a competitively efficient algorithm for implementing SVT-TNN.

In [4]:
def svt_tnn(mat, alpha, rho, theta):
    tau = alpha / rho
    [m, n] = mat.shape
    if 2 * m < n:
        u, s, v = np.linalg.svd(mat @ mat.T, full_matrices = False)
        s = np.sqrt(s)
        idx = np.sum(s > tau)
        mid = np.zeros(idx)
        mid[:theta] = 1
        mid[theta:idx] = (s[theta : idx] - tau) / s[theta : idx]
        return (u[:,:idx] @ np.diag(mid)) @ (u[:, :idx].T @ mat)
    elif m > 2 * n:
        return svt_tnn(mat.T, alpha, rho, theta).T
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    idx = np.sum(s > tau)
    vec = s[:idx].copy()
    vec[theta : idx] = s[theta : idx] - tau
    return u[:, :idx] @ np.diag(vec) @ v[:idx, :]

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>compute_mape</code>:</b> <font color="black">Compute the value of Mean Absolute Percentage Error (MAPE).</font></li>
<li><b><code>compute_rmse</code>:</b> <font color="black">Compute the value of Root Mean Square Error (RMSE).</font></li>
</ul>
</div>

> Note that $$\mathrm{MAPE}=\frac{1}{n} \sum_{i=1}^{n} \frac{\left|y_{i}-\hat{y}_{i}\right|}{y_{i}} \times 100, \quad\mathrm{RMSE}=\sqrt{\frac{1}{n} \sum_{i=1}^{n}\left(y_{i}-\hat{y}_{i}\right)^{2}},$$ where $n$ is the total number of estimated values, and $y_i$ and $\hat{y}_i$ are the actual value and its estimation, respectively.

In [5]:
def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

### Define LRTC-TNN Function with `Numpy`

In [6]:
def LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter):
    """Low-Rank Tenor Completion with Truncated Nuclear Norm, LRTC-TNN."""
    dim = np.array(sparse_tensor.shape)
    pos_missing = np.where(sparse_tensor == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    
    X = np.zeros(np.insert(dim, 0, len(dim))) # \boldsymbol{\mathcal{X}}
    T = np.zeros(np.insert(dim, 0, len(dim))) # \boldsymbol{\mathcal{T}}
    Z = sparse_tensor.copy()
    last_tensor = sparse_tensor.copy()
    snorm = np.sqrt(np.sum(sparse_tensor ** 2))
    it = 0
    while True:
        rho = min(rho * 1.05, 1e5)
        for k in range(len(dim)):
            X[k] = mat2ten(svt_tnn(ten2mat(Z - T[k] / rho, k), alpha[k], rho, np.int(np.ceil(theta * dim[k]))), dim, k)
        Z[pos_missing] = np.mean(X + T / rho, axis = 0)[pos_missing]
        T = T + rho * (X - np.broadcast_to(Z, np.insert(dim, 0, len(dim))))
        tensor_hat = np.einsum('k, kmnt -> mnt', alpha, X)
        tol = np.sqrt(np.sum((tensor_hat - last_tensor) ** 2)) / snorm
        last_tensor = tensor_hat.copy()
        it += 1
        print('Iter: {}'.format(it))
        print('Tolerance: {:.6}'.format(tol))
        print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
        print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
        print()
        if (tol < epsilon) or (it >= maxiter):
            break
    
    print('Total iteration: {}'.format(it))
    print('Tolerance: {:.6}'.format(tol))
    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
    print()
    
    return tensor_hat

In [8]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

start = time.time()
alpha = np.ones(3) / 3
rho = 1e-5
theta = 0.30
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.553985
MAPE: 0.321826
RMSE: 13.5421

Iter: 2
Tolerance: 0.481035
MAPE: 0.183707
RMSE: 6.90845

Iter: 3
Tolerance: 0.0689438
MAPE: 0.222953
RMSE: 8.20973

Iter: 4
Tolerance: 0.111374
MAPE: 0.157234
RMSE: 5.97984

Iter: 5
Tolerance: 0.0819003
MAPE: 0.143751
RMSE: 5.4756

Iter: 6
Tolerance: 0.0188061
MAPE: 0.145837
RMSE: 5.54376

Iter: 7
Tolerance: 0.0171874
MAPE: 0.143576
RMSE: 5.46624

Iter: 8
Tolerance: 0.165942
MAPE: 0.134324
RMSE: 5.65469

Iter: 9
Tolerance: 0.12562
MAPE: 0.160029
RMSE: 6.85603

Iter: 10
Tolerance: 0.208524
MAPE: 0.134461
RMSE: 5.76843

Iter: 11
Tolerance: 0.186198
MAPE: 0.147891
RMSE: 6.11242

Iter: 12
Tolerance: 0.183558
MAPE: 0.170106
RMSE: 7.50426

Iter: 13
Tolerance: 0.24486
MAPE: 0.122516
RMSE: 5.11533

Iter: 14
Tolerance: 0.183071
MAPE: 0.166406
RMSE: 7.24701

Iter: 15
Tolerance: 0.210898
MAPE: 0.132615
RMSE: 5.16953

Iter: 16
Tolerance: 0.119997
MAPE: 0.13901
RMSE: 5.61382

Iter: 17
Tolerance: 0.195039
MAPE: 0.134017
RMSE: 5.78268

Iter: 

In [10]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

start = time.time()
alpha = np.ones(3) / 3
rho = 1e-5
theta = 0.30
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 1.0
MAPE: 1.0
RMSE: 40.486

Iter: 2
Tolerance: 1.06086
MAPE: 0.419618
RMSE: 17.8861

Iter: 3
Tolerance: 0.91566
MAPE: 0.179018
RMSE: 6.83372

Iter: 4
Tolerance: 0.566301
MAPE: 0.480006
RMSE: 17.1711

Iter: 5
Tolerance: 0.174255
MAPE: 0.576447
RMSE: 20.6776

Iter: 6
Tolerance: 0.160295
MAPE: 0.488713
RMSE: 17.4629

Iter: 7
Tolerance: 0.32132
MAPE: 0.306034
RMSE: 11.0476

Iter: 8
Tolerance: 0.33789
MAPE: 0.163258
RMSE: 6.23032

Iter: 9
Tolerance: 0.250301
MAPE: 0.183872
RMSE: 7.04097

Iter: 10
Tolerance: 0.119789
MAPE: 0.220789
RMSE: 8.69015

Iter: 11
Tolerance: 0.0297147
MAPE: 0.217193
RMSE: 8.55532

Iter: 12
Tolerance: 0.0918136
MAPE: 0.1873
RMSE: 7.22997

Iter: 13
Tolerance: 0.118631
MAPE: 0.158874
RMSE: 6.034

Iter: 14
Tolerance: 0.217339
MAPE: 0.128521
RMSE: 5.27145

Iter: 15
Tolerance: 0.192803
MAPE: 0.166683
RMSE: 6.98412

Iter: 16
Tolerance: 0.191586
MAPE: 0.181288
RMSE: 7.74248

Iter: 17
Tolerance: 0.207365
MAPE: 0.164114
RMSE: 6.99953

Iter: 18
Tolerance: 0.2

In [13]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Non-random Missing (NM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

start = time.time()
alpha = np.ones(3) / 3
rho = 1e-5
theta = 0.05
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.37918
MAPE: 0.533876
RMSE: 22.353

Iter: 2
Tolerance: 0.353986
MAPE: 0.165107
RMSE: 6.46655

Iter: 3
Tolerance: 0.20406
MAPE: 0.28282
RMSE: 10.1567

Iter: 4
Tolerance: 0.0563174
MAPE: 0.327222
RMSE: 11.6503

Iter: 5
Tolerance: 0.0736269
MAPE: 0.241519
RMSE: 8.83887

Iter: 6
Tolerance: 0.0883408
MAPE: 0.163517
RMSE: 6.19368

Iter: 7
Tolerance: 0.0619152
MAPE: 0.155775
RMSE: 5.88726

Iter: 8
Tolerance: 0.185033
MAPE: 0.149812
RMSE: 5.77681

Iter: 9
Tolerance: 0.141997
MAPE: 0.158168
RMSE: 6.56669

Iter: 10
Tolerance: 0.207219
MAPE: 0.133853
RMSE: 5.19757

Iter: 11
Tolerance: 0.203058
MAPE: 0.147319
RMSE: 6.14923

Iter: 12
Tolerance: 0.218047
MAPE: 0.151963
RMSE: 6.28579

Iter: 13
Tolerance: 0.233732
MAPE: 0.125168
RMSE: 5.0695

Iter: 14
Tolerance: 0.0981918
MAPE: 0.141322
RMSE: 5.79293

Iter: 15
Tolerance: 0.241205
MAPE: 0.155668
RMSE: 6.63688

Iter: 16
Tolerance: 0.214494
MAPE: 0.139863
RMSE: 5.60026

Iter: 17
Tolerance: 0.234984
MAPE: 0.127191
RMSE: 5.44396

Iter: 

In [15]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

## Non-random Missing (NM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

import time
start = time.time()
alpha = np.ones(3) / 3
rho = 1e-5
theta = 0.05
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 1.0
MAPE: 1.0
RMSE: 40.4194

Iter: 2
Tolerance: 1.20977
MAPE: 0.607054
RMSE: 25.4008

Iter: 3
Tolerance: 0.640007
MAPE: 0.262731
RMSE: 11.3674

Iter: 4
Tolerance: 0.555859
MAPE: 0.252884
RMSE: 9.92456

Iter: 5
Tolerance: 0.376262
MAPE: 0.450062
RMSE: 16.5054

Iter: 6
Tolerance: 0.200597
MAPE: 0.553107
RMSE: 19.8403

Iter: 7
Tolerance: 0.135598
MAPE: 0.539164
RMSE: 19.3619

Iter: 8
Tolerance: 0.19185
MAPE: 0.44325
RMSE: 16.3151

Iter: 9
Tolerance: 0.232838
MAPE: 0.32133
RMSE: 12.388

Iter: 10
Tolerance: 0.231055
MAPE: 0.236936
RMSE: 9.31461

Iter: 11
Tolerance: 0.306719
MAPE: 0.201839
RMSE: 7.96835

Iter: 12
Tolerance: 0.268616
MAPE: 0.210674
RMSE: 8.35135

Iter: 13
Tolerance: 0.11366
MAPE: 0.223274
RMSE: 8.85254

Iter: 14
Tolerance: 0.215304
MAPE: 0.217746
RMSE: 8.71707

Iter: 15
Tolerance: 0.327489
MAPE: 0.193223
RMSE: 8.04524

Iter: 16
Tolerance: 0.221779
MAPE: 0.183803
RMSE: 7.74406

Iter: 17
Tolerance: 0.18173
MAPE: 0.175597
RMSE: 7.44621

Iter: 18
Tolerance: 0.2

In [7]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.3

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Random missing (RM)
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = sparse_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
del dense_mat, sparse_mat

In [8]:
import time
start = time.time()
alpha = np.ones(3) / 3
rho = 1e-5
theta = 0.05
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.594796
MAPE: 0.376063
RMSE: 9.93682

Iter: 2
Tolerance: 0.540654
MAPE: 0.172872
RMSE: 3.8003

Iter: 3
Tolerance: 0.211804
MAPE: 0.2424
RMSE: 5.17444

Iter: 4
Tolerance: 0.115913
MAPE: 0.19401
RMSE: 4.47153

Iter: 5
Tolerance: 0.223191
MAPE: 0.166439
RMSE: 4.20404

Iter: 6
Tolerance: 0.186403
MAPE: 0.122799
RMSE: 3.04614

Iter: 7
Tolerance: 0.122955
MAPE: 0.134474
RMSE: 3.18168

Iter: 8
Tolerance: 0.189031
MAPE: 0.118989
RMSE: 2.96094

Iter: 9
Tolerance: 0.153591
MAPE: 0.126882
RMSE: 3.13082

Iter: 10
Tolerance: 0.13652
MAPE: 0.118765
RMSE: 3.03668

Iter: 11
Tolerance: 0.172052
MAPE: 0.124037
RMSE: 3.11938

Iter: 12
Tolerance: 0.155647
MAPE: 0.130492
RMSE: 3.15162

Iter: 13
Tolerance: 0.159646
MAPE: 0.123041
RMSE: 3.03678

Iter: 14
Tolerance: 0.163752
MAPE: 0.118314
RMSE: 2.93933

Iter: 15
Tolerance: 0.146654
MAPE: 0.11701
RMSE: 2.88581

Iter: 16
Tolerance: 0.13477
MAPE: 0.115592
RMSE: 2.80077

Iter: 17
Tolerance: 0.11093
MAPE: 0.107163
RMSE: 2.60861

Iter: 18
Toler

In [9]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.7

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Random missing (RM)
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = sparse_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
del dense_mat, sparse_mat

In [10]:
import time
start = time.time()
alpha = np.ones(3) / 3
rho = 1e-5
theta = 0.05
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.83619
MAPE: 0.723064
RMSE: 18.0079

Iter: 2
Tolerance: 0.706661
MAPE: 0.36246
RMSE: 9.59087

Iter: 3
Tolerance: 0.664945
MAPE: 0.172871
RMSE: 4.04806

Iter: 4
Tolerance: 0.481564
MAPE: 0.309997
RMSE: 6.6263

Iter: 5
Tolerance: 0.259395
MAPE: 0.41343
RMSE: 8.79255

Iter: 6
Tolerance: 0.137026
MAPE: 0.399503
RMSE: 8.60932

Iter: 7
Tolerance: 0.190635
MAPE: 0.307362
RMSE: 6.95946

Iter: 8
Tolerance: 0.262789
MAPE: 0.196155
RMSE: 4.99706

Iter: 9
Tolerance: 0.245024
MAPE: 0.166012
RMSE: 4.25843

Iter: 10
Tolerance: 0.208759
MAPE: 0.165443
RMSE: 4.15261

Iter: 11
Tolerance: 0.174727
MAPE: 0.16892
RMSE: 4.21834

Iter: 12
Tolerance: 0.151707
MAPE: 0.1504
RMSE: 3.79153

Iter: 13
Tolerance: 0.182851
MAPE: 0.132283
RMSE: 3.34923

Iter: 14
Tolerance: 0.165312
MAPE: 0.121576
RMSE: 3.06316

Iter: 15
Tolerance: 0.152098
MAPE: 0.130689
RMSE: 3.23769

Iter: 16
Tolerance: 0.148996
MAPE: 0.130396
RMSE: 3.1909

Iter: 17
Tolerance: 0.168163
MAPE: 0.130411
RMSE: 3.12465

Iter: 18
Toler

In [12]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.3

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Non-random missing (NM)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, 30) + 0.5 - missing_rate)[:, None, :]
del dense_mat

In [13]:
import time
start = time.time()
alpha = np.ones(3) / 3
rho = 1e-5
theta = 0.05
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.462486
MAPE: 0.575285
RMSE: 14.6067

Iter: 2
Tolerance: 0.431005
MAPE: 0.183526
RMSE: 4.7571

Iter: 3
Tolerance: 0.274165
MAPE: 0.253887
RMSE: 5.47007

Iter: 4
Tolerance: 0.159628
MAPE: 0.335471
RMSE: 7.22953

Iter: 5
Tolerance: 0.102257
MAPE: 0.261863
RMSE: 6.23517

Iter: 6
Tolerance: 0.190787
MAPE: 0.176908
RMSE: 4.2935

Iter: 7
Tolerance: 0.183531
MAPE: 0.132081
RMSE: 3.18496

Iter: 8
Tolerance: 0.144973
MAPE: 0.12345
RMSE: 3.08101

Iter: 9
Tolerance: 0.150581
MAPE: 0.129057
RMSE: 3.18794

Iter: 10
Tolerance: 0.149484
MAPE: 0.13117
RMSE: 3.25158

Iter: 11
Tolerance: 0.185699
MAPE: 0.123085
RMSE: 3.06191

Iter: 12
Tolerance: 0.157056
MAPE: 0.120778
RMSE: 2.94206

Iter: 13
Tolerance: 0.152408
MAPE: 0.126825
RMSE: 3.14873

Iter: 14
Tolerance: 0.15411
MAPE: 0.11625
RMSE: 2.84266

Iter: 15
Tolerance: 0.136577
MAPE: 0.121619
RMSE: 3.01488

Iter: 16
Tolerance: 0.155688
MAPE: 0.120687
RMSE: 2.90498

Iter: 17
Tolerance: 0.112422
MAPE: 0.114617
RMSE: 2.79377

Iter: 18
Tol

In [14]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.7

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Non-random missing (NM)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, 30) + 0.5 - missing_rate)[:, None, :]
del dense_mat

In [15]:
import time
start = time.time()
alpha = np.ones(3) / 3
rho = 1e-5
theta = 0.05
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.61093
MAPE: 0.815609
RMSE: 20.0953

Iter: 2
Tolerance: 0.513516
MAPE: 0.532671
RMSE: 13.7514

Iter: 3
Tolerance: 0.541258
MAPE: 0.277075
RMSE: 7.67951

Iter: 4
Tolerance: 0.476985
MAPE: 0.224449
RMSE: 5.36902

Iter: 5
Tolerance: 0.386449
MAPE: 0.326423
RMSE: 7.25669

Iter: 6
Tolerance: 0.261234
MAPE: 0.414577
RMSE: 8.97826

Iter: 7
Tolerance: 0.194105
MAPE: 0.418278
RMSE: 9.32123

Iter: 8
Tolerance: 0.189291
MAPE: 0.375928
RMSE: 8.64799

Iter: 9
Tolerance: 0.230311
MAPE: 0.304162
RMSE: 7.44639

Iter: 10
Tolerance: 0.246236
MAPE: 0.239882
RMSE: 6.20592

Iter: 11
Tolerance: 0.21578
MAPE: 0.201786
RMSE: 5.2982

Iter: 12
Tolerance: 0.197809
MAPE: 0.183872
RMSE: 4.78763

Iter: 13
Tolerance: 0.181371
MAPE: 0.170779
RMSE: 4.40921

Iter: 14
Tolerance: 0.170338
MAPE: 0.166571
RMSE: 4.26474

Iter: 15
Tolerance: 0.201802
MAPE: 0.161258
RMSE: 4.13143

Iter: 16
Tolerance: 0.177015
MAPE: 0.150231
RMSE: 3.85023

Iter: 17
Tolerance: 0.158831
MAPE: 0.149444
RMSE: 3.80018

Iter: 18


## Missing Data Imputation

In the following, we apply the above defined LRTC-TNN function to the task of missing data imputation on the following spatiotemporal multivariate time series datasets/matrices:

- **PeMS data set**: [PeMS traffic speed data set](https://doi.org/10.5281/zenodo.3939793).

### PeMS-4W

We generate **random missing (RM)** values on PeMS data set.

In [8]:
import time
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 4 * 7)

missing_rate = 0.3

### Random missing (RM) scenario:
sparse_tensor = np.multiply(dense_tensor, np.round(random_tensor + 0.5 - missing_rate))
del data, random_tensor

start = time.time()
alpha = np.ones(3) / 3
rho = 1e-4
theta = 0.05
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.181052
MAPE: 0.782707
RMSE: 50.2879

Iter: 2
Tolerance: 0.195387
MAPE: 0.560058
RMSE: 36.5057

Iter: 3
Tolerance: 0.168308
MAPE: 0.359063
RMSE: 24.2606

Iter: 4
Tolerance: 0.145435
MAPE: 0.184398
RMSE: 13.4393

Iter: 5
Tolerance: 0.115872
MAPE: 0.0784202
RMSE: 6.07173

Iter: 6
Tolerance: 0.0789183
MAPE: 0.0665844
RMSE: 4.76338

Iter: 7
Tolerance: 0.0458254
MAPE: 0.0847863
RMSE: 5.63931

Iter: 8
Tolerance: 0.0367108
MAPE: 0.083127
RMSE: 5.47457

Iter: 9
Tolerance: 0.0397892
MAPE: 0.0670385
RMSE: 4.48838

Iter: 10
Tolerance: 0.0411403
MAPE: 0.0468935
RMSE: 3.30249

Iter: 11
Tolerance: 0.0403542
MAPE: 0.0317915
RMSE: 2.43241

Iter: 12
Tolerance: 0.038941
MAPE: 0.0248937
RMSE: 2.03359

Iter: 13
Tolerance: 0.0377077
MAPE: 0.0231981
RMSE: 1.92444

Iter: 14
Tolerance: 0.0363331
MAPE: 0.0224476
RMSE: 1.87562

Iter: 15
Tolerance: 0.035165
MAPE: 0.0219749
RMSE: 1.8485

Iter: 16
Tolerance: 0.0342945
MAPE: 0.0214996
RMSE: 1.82146

Iter: 17
Tolerance: 0.0333032
MAPE: 0.0212269


In [10]:
import time
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 4 * 7)

missing_rate = 0.7

### Random missing (RM) scenario:
sparse_tensor = np.multiply(dense_tensor, np.round(random_tensor + 0.5 - missing_rate))
del data, random_tensor

start = time.time()
alpha = np.ones(3) / 3
rho = 1e-4
theta = 0.05
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.276156
MAPE: 0.906166
RMSE: 58.0419

Iter: 2
Tolerance: 0.27572
MAPE: 0.808607
RMSE: 52.1791

Iter: 3
Tolerance: 0.189472
MAPE: 0.71773
RMSE: 46.9454

Iter: 4
Tolerance: 0.17281
MAPE: 0.633354
RMSE: 42.0687

Iter: 5
Tolerance: 0.155918
MAPE: 0.557491
RMSE: 37.7344

Iter: 6
Tolerance: 0.143533
MAPE: 0.488646
RMSE: 33.6451

Iter: 7
Tolerance: 0.130831
MAPE: 0.427656
RMSE: 29.9211

Iter: 8
Tolerance: 0.122855
MAPE: 0.372287
RMSE: 26.3898

Iter: 9
Tolerance: 0.113134
MAPE: 0.322893
RMSE: 23.1531

Iter: 10
Tolerance: 0.106687
MAPE: 0.277548
RMSE: 20.1011

Iter: 11
Tolerance: 0.0987406
MAPE: 0.236777
RMSE: 17.3062

Iter: 12
Tolerance: 0.0934201
MAPE: 0.199142
RMSE: 14.6872

Iter: 13
Tolerance: 0.0870086
MAPE: 0.165301
RMSE: 12.3019

Iter: 14
Tolerance: 0.0824562
MAPE: 0.134294
RMSE: 10.0958

Iter: 15
Tolerance: 0.0765782
MAPE: 0.106987
RMSE: 8.13265

Iter: 16
Tolerance: 0.0705283
MAPE: 0.0832733
RMSE: 6.41103

Iter: 17
Tolerance: 0.0633333
MAPE: 0.064051
RMSE: 4.98369

I

We generate **non-random missing (NM)** values on PeMS data set. Then, we conduct the imputation experiment.

In [12]:
import time
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

start = time.time()
alpha = np.ones(3) / 3
rho = 1e-5
theta = 0.05
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.125128
MAPE: 0.867413
RMSE: 55.4783

Iter: 2
Tolerance: 0.122031
MAPE: 0.74696
RMSE: 47.6661

Iter: 3
Tolerance: 0.103463
MAPE: 0.630497
RMSE: 40.3281

Iter: 4
Tolerance: 0.149923
MAPE: 0.450316
RMSE: 29.4888

Iter: 5
Tolerance: 0.142983
MAPE: 0.328168
RMSE: 21.6391

Iter: 6
Tolerance: 0.130813
MAPE: 0.21113
RMSE: 14.9015

Iter: 7
Tolerance: 0.112119
MAPE: 0.109724
RMSE: 9.14

Iter: 8
Tolerance: 0.0940634
MAPE: 0.0820072
RMSE: 7.15026

Iter: 9
Tolerance: 0.0820413
MAPE: 0.0894765
RMSE: 6.87678

Iter: 10
Tolerance: 0.0864565
MAPE: 0.0843179
RMSE: 6.15381

Iter: 11
Tolerance: 0.0910586
MAPE: 0.0700316
RMSE: 5.31845

Iter: 12
Tolerance: 0.0938788
MAPE: 0.0609626
RMSE: 4.8927

Iter: 13
Tolerance: 0.093614
MAPE: 0.060151
RMSE: 4.86196

Iter: 14
Tolerance: 0.0937059
MAPE: 0.0603958
RMSE: 4.88068

Iter: 15
Tolerance: 0.0930535
MAPE: 0.0605985
RMSE: 4.90473

Iter: 16
Tolerance: 0.0960272
MAPE: 0.0591932
RMSE: 4.84462

Iter: 17
Tolerance: 0.0949935
MAPE: 0.0594277
RMSE: 4.8

In [20]:
import time
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

start = time.time()
alpha = np.ones(3) / 3
rho = 1e-6
theta = 0.05
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.670918
MAPE: 1.0
RMSE: 63.7516

Iter: 2
Tolerance: 0.740001
MAPE: 0.619773
RMSE: 40.608

Iter: 3
Tolerance: 0.633251
MAPE: 0.272521
RMSE: 19.7281

Iter: 4
Tolerance: 0.544303
MAPE: 0.241767
RMSE: 16.2671

Iter: 5
Tolerance: 0.371342
MAPE: 0.395773
RMSE: 24.6338

Iter: 6
Tolerance: 0.220667
MAPE: 0.486655
RMSE: 29.2177

Iter: 7
Tolerance: 0.170346
MAPE: 0.478699
RMSE: 28.8084

Iter: 8
Tolerance: 0.199537
MAPE: 0.400068
RMSE: 25.1451

Iter: 9
Tolerance: 0.221094
MAPE: 0.303136
RMSE: 20.3761

Iter: 10
Tolerance: 0.214256
MAPE: 0.233673
RMSE: 16.3231

Iter: 11
Tolerance: 0.187391
MAPE: 0.20348
RMSE: 13.9549

Iter: 12
Tolerance: 0.254431
MAPE: 0.191135
RMSE: 12.8224

Iter: 13
Tolerance: 0.229579
MAPE: 0.191378
RMSE: 12.543

Iter: 14
Tolerance: 0.106563
MAPE: 0.186386
RMSE: 12.2378

Iter: 15
Tolerance: 0.18921
MAPE: 0.171756
RMSE: 11.5291

Iter: 16
Tolerance: 0.243535
MAPE: 0.150097
RMSE: 10.6339

Iter: 17
Tolerance: 0.130325
MAPE: 0.137108
RMSE: 9.74656

Iter: 18
Tolera

### PeMS-8W

We generate **random missing (RM)** values on PeMS data set.

In [16]:
import time
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 8 * 7)

missing_rate = 0.3

### Random missing (RM) scenario:
sparse_tensor = np.multiply(dense_tensor, np.round(random_tensor + 0.5 - missing_rate))
del data, random_tensor

start = time.time()
alpha = np.ones(3) / 3
rho = 1e-4
theta = 0.05
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.15968
MAPE: 0.806788
RMSE: 51.8224

Iter: 2
Tolerance: 0.165518
MAPE: 0.614269
RMSE: 39.8896

Iter: 3
Tolerance: 0.14052
MAPE: 0.435995
RMSE: 28.9255

Iter: 4
Tolerance: 0.129915
MAPE: 0.27316
RMSE: 18.8238

Iter: 5
Tolerance: 0.111815
MAPE: 0.141717
RMSE: 10.4335

Iter: 6
Tolerance: 0.0891459
MAPE: 0.0620955
RMSE: 4.68434

Iter: 7
Tolerance: 0.0598619
MAPE: 0.0621202
RMSE: 4.32664

Iter: 8
Tolerance: 0.0406658
MAPE: 0.0797526
RMSE: 5.25488

Iter: 9
Tolerance: 0.034345
MAPE: 0.0791059
RMSE: 5.19637

Iter: 10
Tolerance: 0.0349264
MAPE: 0.0653536
RMSE: 4.36703

Iter: 11
Tolerance: 0.0356323
MAPE: 0.0473857
RMSE: 3.30191

Iter: 12
Tolerance: 0.0343215
MAPE: 0.0322715
RMSE: 2.44754

Iter: 13
Tolerance: 0.0321294
MAPE: 0.0241542
RMSE: 2.00267

Iter: 14
Tolerance: 0.0303282
MAPE: 0.0214112
RMSE: 1.83168

Iter: 15
Tolerance: 0.0289342
MAPE: 0.0208294
RMSE: 1.7797

Iter: 16
Tolerance: 0.0279787
MAPE: 0.0201668
RMSE: 1.73811

Iter: 17
Tolerance: 0.0267485
MAPE: 0.0197632
RM

In [18]:
import time
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 8 * 7)

missing_rate = 0.7

### Random missing (RM) scenario:
sparse_tensor = np.multiply(dense_tensor, np.round(random_tensor + 0.5 - missing_rate))
del data, random_tensor

start = time.time()
alpha = np.ones(3) / 3
rho = 1e-4
theta = 0.05
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.243622
MAPE: 0.916532
RMSE: 58.7165

Iter: 2
Tolerance: 0.24178
MAPE: 0.832558
RMSE: 53.6622

Iter: 3
Tolerance: 0.154562
MAPE: 0.753982
RMSE: 49.0774

Iter: 4
Tolerance: 0.141411
MAPE: 0.680861
RMSE: 44.815

Iter: 5
Tolerance: 0.128172
MAPE: 0.614093
RMSE: 40.9515

Iter: 6
Tolerance: 0.117644
MAPE: 0.552855
RMSE: 37.3174

Iter: 7
Tolerance: 0.107875
MAPE: 0.497379
RMSE: 33.9646

Iter: 8
Tolerance: 0.100869
MAPE: 0.446579
RMSE: 30.7926

Iter: 9
Tolerance: 0.0933413
MAPE: 0.400409
RMSE: 27.8518

Iter: 10
Tolerance: 0.0874642
MAPE: 0.357856
RMSE: 25.0786

Iter: 11
Tolerance: 0.0810908
MAPE: 0.318921
RMSE: 22.5035

Iter: 12
Tolerance: 0.0760835
MAPE: 0.282835
RMSE: 20.0809

Iter: 13
Tolerance: 0.0708531
MAPE: 0.249724
RMSE: 17.8325

Iter: 14
Tolerance: 0.0667261
MAPE: 0.218991
RMSE: 15.7221

Iter: 15
Tolerance: 0.0623536
MAPE: 0.190825
RMSE: 13.7691

Iter: 16
Tolerance: 0.058876
MAPE: 0.164751
RMSE: 11.9452

Iter: 17
Tolerance: 0.0551433
MAPE: 0.140958
RMSE: 10.2688



We generate **non-random missing (NM)** values on PeMS data set. Then, we conduct the imputation experiment.

In [22]:
import time
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

start = time.time()
alpha = np.ones(3) / 3
rho = 1e-5
theta = 0.05
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.12317
MAPE: 0.867673
RMSE: 55.3517

Iter: 2
Tolerance: 0.115052
MAPE: 0.744081
RMSE: 47.3842

Iter: 3
Tolerance: 0.101592
MAPE: 0.626195
RMSE: 39.7447

Iter: 4
Tolerance: 0.148729
MAPE: 0.441145
RMSE: 28.6189

Iter: 5
Tolerance: 0.143841
MAPE: 0.316507
RMSE: 20.6786

Iter: 6
Tolerance: 0.129217
MAPE: 0.196263
RMSE: 13.5911

Iter: 7
Tolerance: 0.106335
MAPE: 0.0960332
RMSE: 7.94286

Iter: 8
Tolerance: 0.0901724
MAPE: 0.0755248
RMSE: 6.05062

Iter: 9
Tolerance: 0.0826911
MAPE: 0.0846522
RMSE: 5.95276

Iter: 10
Tolerance: 0.0822369
MAPE: 0.0757313
RMSE: 5.48285

Iter: 11
Tolerance: 0.0836934
MAPE: 0.063912
RMSE: 4.9838

Iter: 12
Tolerance: 0.0858154
MAPE: 0.0589949
RMSE: 4.78313

Iter: 13
Tolerance: 0.0869298
MAPE: 0.0590947
RMSE: 4.78225

Iter: 14
Tolerance: 0.0882782
MAPE: 0.0593871
RMSE: 4.80297

Iter: 15
Tolerance: 0.0873945
MAPE: 0.0594621
RMSE: 4.82892

Iter: 16
Tolerance: 0.0876289
MAPE: 0.0589578
RMSE: 4.831

Iter: 17
Tolerance: 0.0878311
MAPE: 0.0578941
RMSE:

In [24]:
import time
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

start = time.time()
alpha = np.ones(3) / 3
rho = 1e-6
theta = 0.05
epsilon = 1e-3
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.671375
MAPE: 1.0
RMSE: 63.8232

Iter: 2
Tolerance: 0.723968
MAPE: 0.611444
RMSE: 39.9236

Iter: 3
Tolerance: 0.645231
MAPE: 0.246217
RMSE: 16.9267

Iter: 4
Tolerance: 0.562335
MAPE: 0.219779
RMSE: 14.2832

Iter: 5
Tolerance: 0.380154
MAPE: 0.422896
RMSE: 25.4146

Iter: 6
Tolerance: 0.199423
MAPE: 0.526677
RMSE: 30.9369

Iter: 7
Tolerance: 0.132041
MAPE: 0.512379
RMSE: 30.1968

Iter: 8
Tolerance: 0.192578
MAPE: 0.414108
RMSE: 25.2171

Iter: 9
Tolerance: 0.290839
MAPE: 0.281561
RMSE: 18.5201

Iter: 10
Tolerance: 0.28553
MAPE: 0.193033
RMSE: 13.2302

Iter: 11
Tolerance: 0.246134
MAPE: 0.171317
RMSE: 11.2589

Iter: 12
Tolerance: 0.245045
MAPE: 0.174887
RMSE: 11.5234

Iter: 13
Tolerance: 0.13918
MAPE: 0.193542
RMSE: 12.3715

Iter: 14
Tolerance: 0.124861
MAPE: 0.186032
RMSE: 12.2063

Iter: 15
Tolerance: 0.160521
MAPE: 0.166248
RMSE: 11.3429

Iter: 16
Tolerance: 0.191212
MAPE: 0.141983
RMSE: 10.1123

Iter: 17
Tolerance: 0.167704
MAPE: 0.122834
RMSE: 8.95309

Iter: 18
Tole

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>